# Fetch Data

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('../')))
import pdal
import json
from scripts.data_fetcher import FetchData
import pprint
pp = pprint.PrettyPrinter(indent=14)


## Explore the data 
Taking a sample laz and ept.json file to explore what is inside

In [2]:
with open("data/ept.json",'r') as ept_file:
    ept_json = json.load(ept_file)

pp.pprint(ept_json)


{             'bounds': [-11752672, 4740364, -68269, -11610700, 4882336, 73703],
              'boundsConforming': [             -11752670,
                                                4750545,
                                                1136,
                                                -11610701,
                                                4872154,
                                                4297],
              'dataType': 'laszip',
              'hierarchyType': 'json',
              'points': 33711288742,
              'schema': [             {             'name': 'X',
                                                    'offset': -11681686,
                                                    'scale': 0.001,
                                                    'size': 4,
                                                    'type': 'signed'},
                                      {             'name': 'Y',
                                                    'offset':

#### We can see from the json file that the ept.json file is indexed with 33711288742 points. The data is compressed using laszip which is a laz file format.

In [3]:
import numpy as np
import pylas

with pylas.open('data/6-50-20-32.laz') as fh:
    print('Points from Header:', fh.header.point_count)
    las = fh.read()
    print(las)
    print('Points from data:', len(las.points))
    ground_pts = las.classification == 2
    bins, counts = np.unique(las.return_number[ground_pts], return_counts=True)
    print('Ground Point Return Number distribution:')
    for r,c in zip(bins,counts):
        print('    {}:{}'.format(r,c))

Points from Header: 46853
<LasData(1.2, point fmt: <PointFormat(1)>, 46853 points, 3 vlrs)>
Points from data: 46853
Ground Point Return Number distribution:
    1:20731
    2:779
    3:25
    4:2


## Define the Pipeline

In [4]:
pipeline_def = [
    {
        "type": "readers.ept",
        "bounds": "",
        "filename": "",
        "tag":"read_data"
    },
    {
        "type":"writers.text",
        "format":"geojson",
        "order":"X,Y,Z",
        "keep_unspecified":"false",
        "filename":"outputfile.txt"
    }
]

In [ ]:
fd = FetchData(public_data_path = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/",
region = 'USGS_LPC_CO_SoPlatteRiver_Lot5_2013_LAS_2015/',pipeline_def= pipeline_def,
bounds = "([-11669524.7,-11666600],[4776607.3,4778714])"
)

